In [1]:
import pandas as pd

# Prepare gos data

In [2]:
df_gos = pd.read_excel('data/grls2018-03-16-1.xls')
df_gos_place = pd.read_csv('data/df_gos.csv', encoding = 'cp1251', index_col = 0)
df_gos_place.drop('(0, \'place_id\')', 1, inplace = True)
df_gos_place.rename(columns = {'name':'Юридическое лицо, на имя которого выдано регистрационное удостоверение'}, inplace = True)
df_gos = df_gos.merge(df_gos_place, on="Юридическое лицо, на имя которого выдано регистрационное удостоверение", how = 'left' )

In [9]:
df_gos[['place_id', 'Юридическое лицо, на имя которого выдано регистрационное удостоверение', 'Торговое наименование\nлекарственного препарата']].drop_duplicates().to_csv('data/gos_dict.csv', sep=';', index = False)

In [9]:
# # save gos dict for later use
# df_gos[['place_id', 'Юридическое лицо, на имя которого выдано регистрационное удостоверение', 'Торговое наименование\nлекарственного препарата']].drop_duplicates().to_csv('data/gos_dict.csv', sep=';', index = False)

In [4]:
# # save gos dict for later use
# df_gos[['place_id', 'Юридическое лицо, на имя которого выдано регистрационное удостоверение']].drop_duplicates().to_csv('data/gos_dict_manuf.csv', sep=';', index = False)

In [15]:
# stats
df_gos['Дата регистрации'] = pd.DatetimeIndex(df_gos['Дата регистрации'])
df_gos['Дата окончания действия регистрационного удостоверения'] = pd.DatetimeIndex(df_gos['Дата окончания действия регистрационного удостоверения'])
df_gos['Дата аннулирования регистрационного удостоверения'] = pd.DatetimeIndex(df_gos['Дата аннулирования регистрационного удостоверения'])
df_gos['place_was_nullated'] = pd.isnull(df_gos['Дата аннулирования регистрационного удостоверения'])
df_gos['place_no_end'] = pd.isnull(df_gos['Дата окончания действия регистрационного удостоверения'])
df_gos['place_time'] = (df_gos['Дата окончания действия регистрационного удостоверения'] - df_gos['Дата регистрации']).dt.days

In [16]:
# stats by place
df_gos = df_gos.groupby(['place_id', 'Юридическое лицо, на имя которого выдано регистрационное удостоверение'], as_index = False).agg({'place_was_nullated':'sum', 'place_no_end':'sum', 'place_time':'mean', 'Номер регистрационного удостоверения':'size'}).rename(columns={'Номер регистрационного удостоверения':'place_size'})
df_gos['place_was_nullated'] = df_gos['place_was_nullated'] / df_gos['place_size']
df_gos['place_no_end'] = df_gos['place_no_end'] / df_gos['place_size']
df_gos.head(1)

,place_id,"Юридическое лицо, на имя которого выдано регистрационное удостоверение",place_was_nullated,place_no_end,place_time,place_size
0,ChIJ-4GgOlNKtUYRJAfJskWS2Sc,Стерлинг С.п.А.,1.0,1.0,NaN,1


# Save gos data

In [21]:
df_gos = df_gos.groupby(['place_id'], as_index = False).agg({'place_was_nullated':'mean', 'place_no_end':'mean', 'place_time':'mean', 'place_size':'sum', 'Юридическое лицо, на имя которого выдано регистрационное удостоверение':'max'})
df_gos.to_csv('data/gos_prepared.csv', encoding='cp1251', sep=';', index = False)